In [1]:
import duckdb
from dotenv import load_dotenv
import os
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain import HuggingFaceHub, OpenAI
from langchain import PromptTemplate, LLMChain
from sqlalchemy import Column, Integer, Sequence, String, create_engine
from langchain.llms import AzureOpenAI


duckdb.__version__

load_dotenv()
# OPENAI_API_KEY = os.getenv('OPENAI_API_TYPE')
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_API_TYPE = os.getenv("OPENAI_API_TYPE")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")
OPENAI_API_BASE = os.getenv("OPENAI_API_BASE")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL_NAME = "text-davinci-003"
DEPLOYMENT_NAME= os.getenv("DEPLOYMENT_NAME")

uri = 'duckdb:///db/cdn_open_data.db'

connect_args = {
    'read_only': True
}

CONN = create_engine(uri)

# Create an instance of Azure OpenAI
# Replace the deployment name with your own
# Initiate a connection to the LLM from Azure OpenAI Service via LangChain.
# llm = AzureOpenAI(
#     openai_api_key=OPENAI_API_KEY,
#     deployment_name=DEPLOYMENT_NAME,
#     model_name=DEPLOYMENT_NAME, 
# )

# Create an instance of Azure OpenAI
# Replace the deployment name with your own
llm = AzureOpenAI(
    deployment_name=DEPLOYMENT_NAME,
    model_name=DEPLOYMENT_NAME,
)

# Make a simple prompt to validate the connection with LLM from Azure OpenAI Service.
print(llm("Tell me a joke"))

# db = SQLDatabase.from_uri(
#     uri,
#     include_tables=['inventory'], 
# 	sample_rows_in_table_info=3)

# toolkit = SQLDatabaseToolkit(db=db, llm=AzureOpenAI(temperature=0))

# agent_executor = create_sql_agent(
#     llm=AzureOpenAI(temperature=0),
#     toolkit=toolkit,
#     verbose=True,
#     agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
# )

# template = """/
# You are a SQL Analyst that is querying a database of Canada Open Data Inventory that about all the Canada Open Data from the Government of Canada website.

# Below is a description of the columns, data types, and information in the columns:

# The column name ref_number with the data type VARCHAR contains the following information: Unique identifier for every open data
# The column name title_en with the data type VARCHAR contains the following information: English title of the open data
# The column name title_fr with the data type VARCHAR contains the following information: French title of the open data
# The column name description_en with the data type VARCHAR contains the following information: English description of the open data
# The column name description_fr with the data type VARCHAR contains the following information: French description of the open data
# The column name publisher_en with the data type VARCHAR contains the following information: Publisher name in English
# The column name publisher_fr with the data type VARCHAR contains the following information: Publisher name in French
# The column name date_published with the data type VARCHAR contains the following information: The date of this open data published
# The column name language with the data type VARCHAR contains the following information: What language this open data
# The column name size with the data type BIGINT contains the following information: The open data size
# The column name program_alignment_architecture_en with the data type VARCHAR contains the following information: English name of the program alignment architecture
# The column name program_alignment_architecture_fr with the data type VARCHAR contains the following information: French name of the program alignment architecture
# The column name date_released with the data type VARCHAR contains the following information: The date this open data released
# The column name portal_url_en with the data type VARCHAR contains the following information: English portal url 
# The column name portal_url_fr with the data type VARCHAR contains the following information: French portal url 
# The column name user_votes with the data type BIGINT contains the following information: The users votes count for this open data, which showing which open data are most request by the user
# The column name owner_org with the data type VARCHAR contains the following information: Which org divison are the owner of this open data
# The column name owner_org_title with the data type VARCHAR contains the following information: The org division title of the owner of this open data

# Your job is to write an execute a query that answers the following question:
# {query}
# """

# prompt = PromptTemplate.from_template(template)

# agent_executor.run(
#     prompt.format(query = "What's total open data record?")
# )


/Users/huangwh/python/duckdb-llm-openai-demo/.venv/lib/python3.10/site-packages/langchain/__init__.py:24: UserWarning: Importing HuggingFaceHub from langchain root module is no longer supported.
  warnings.warn(
/Users/huangwh/python/duckdb-llm-openai-demo/.venv/lib/python3.10/site-packages/langchain/__init__.py:24: UserWarning: Importing OpenAI from langchain root module is no longer supported.
  warnings.warn(
/Users/huangwh/python/duckdb-llm-openai-demo/.venv/lib/python3.10/site-packages/langchain/__init__.py:24: UserWarning: Importing PromptTemplate from langchain root module is no longer supported.
  warnings.warn(
/Users/huangwh/python/duckdb-llm-openai-demo/.venv/lib/python3.10/site-packages/langchain/__init__.py:24: UserWarning: Importing LLMChain from langchain root module is no longer supported.
  warnings.warn(


](https://docs.microsoft.com/en-us/azure/cognitive-services/openai-service/)
- [Tutorial: Use OpenAI to generate text](https://docs.microsoft.com/en-us/azure/cognitive-services/openai-service/get-started-openai-text-generation-tutorial)

## Prerequisites

- An [Azure subscription](https://azure.microsoft.com/free/).
- [Python 3](https://www.python.org/downloads/).
- [Azure CLI](https://docs.microsoft.com/cli/azure/install-azure-cli?view=azure-cli-latest).
- [Postman](https://www.postman.com/downloads/).

## Create an Azure OpenAI Service resource

1. In the Azure portal, select **Create a resource**.

2. In the **New page**, search for **OpenAI Service**. 

3. Choose **OpenAI Service** from the search result.

4. In the **OpenAI Service** page, select **Create**.

5. In the **Basics** tab, enter the following configuration details:

   - **Subscription**: Choose your subscription.

   - **Resource group**: Choose a resource group or create a new one.

   - **Region**: Choose a region.
